# Assignment 5 - Pathfinder

Welcome to Assignment 5 of the "Robot Programming with Lisp" course.

* Due Date: 30.11.2022, 23:59 CEST in your Github repository
* 10 points achievable

We are back in the simulation with our well known maze. In the previous assignments you
teleported the robot to each treasure directly. Now the robot moves and turns step by step.

The 10 treasures are colored :RED and :BLUE. There are 2 depots in the world, one is
:RED, the other one :BLUE. The depots are stored in a hash-table of the treasure-world.
The keys to the depots in the hash-table are :RED and :BLUE. Get the respective entity with `(gethash :RED (depots *world*))`.

The robot can carry up to 2 treasures in his trunk. Get the trunk slots like this: `(aref (trunk robot) 0)`

Your task is to bring the treasures to the depot of their respective color.
For that, you must find a step-by-step path to each target recursively.


## How to

Execute the code segments one after another. Each chapter in this notebook serves its purpose.

* **Load Package** - How to load the package of the assignment
* **Prepared code** - Some prepared code, containing parameters and functions for the world. Don't touch that.
* **Assignment code** - Your assignment. This is where you need to write code. Solve the missing TODOs by implementing the functionality described.
* **Testing** - With these functions you can test your program. Add more code blocks here if you want to test specific functions.

You have completed the assignment when the robot picks up all the treasures in the world. Your grade depends on your individual implementation in the **Assignment code**.

## Load Package

Execute the code of the following section to load this assignments code.

In [ ]:
;; Adds the directory of this assignment into the ASDF registry
(let* ((lecture-path '(:absolute "home" "lectures" "robot_programming_with_lisp"))
       (assignment-path (append lecture-path '("05_pathfinder")))
       (bullet-path (append lecture-path '("bullet_wrapper"))))
      (pushnew (make-pathname :directory assignment-path) asdf:*central-registry*)
      (pushnew (make-pathname :directory bullet-path) asdf:*central-registry*))

In [ ]:
;; Loads the ASDF system of this assignment
(asdf:load-system :pathfinder)

In [ ]:
;; Makes this assignment as the current namespace
(in-package :pathfinder)

## Prepared Code

The oop-world from the last assignment is already implemented.

In [ ]:
(defparameter *world* (initialize-world))

## Assignment Code - Robot Actions

This code focuses on the actions of the robot: move, collect, deposit

In [ ]:
(defgeneric valid-move (robot x y orientation)
  (:documentation "Checks the attempted move. The following operations are valid:
                  a) change the orientation while x and y are the robot's coordinates.
                  b) the robot's pose doesn't change.
                  c) take one step towards the current orientation without changing the orientation.
                  In other words: either turn, stay or make a step forward.")
  (:method ((robot robot) x y orientation)
    ;; TODO implement
    T))

In [ ]:
(defgeneric valid-coord (x y world)
  (:documentation "Checks if the coordinates are within bounds (15x16) and unoccupied 
                  by walls, treasures or depots.")
  (:method (x y (world treasure-world))
    ;; TODO implement
    T))

In [ ]:
(defgeneric collect-treasure (robot)
  (:documentation "Collects the treasure laying in front of the robot, if there is any,
                  and puts it into a free slot in his trunk.")
  (:method ((robot robot))
    ;; TODO implement
    ))

In [ ]:
(defgeneric deposit-treasure (robot)
  (:documentation "While standing in front of a depot, removes all treasures in the trunk
                  that match the color of the depot.")
  (:method ((robot robot))
    ;; TODO implement
    ))

In [ ]:
(defgeneric move (robot x y &optional orientation)
  (:documentation "Moves the robot if `valid-move' approves. Also counts the total of moves made."))

;; TODO Embed the move method in a closure. Within this closure, increment a local variable
;;      for each successful move made. No defparameter, defvar etc. allowed, the variable
;;      must not be visible from the outside.
;; TODO Within the closure, alongside the move method, define a function that returns the
;;      value of the local movement counter.
;;
;; The goal is to monitor the amount of steps we made after taking a path, or multiple paths.
;; The variable resets with every compilation of the code.
(defmethod move ((robot robot) x y &optional orientation)
  (if (not (valid-move robot x y orientation))
      (warn "The action is invalid. Either turn the robot or move forward.")
      (when (valid-coord x y (world robot))
          (setf (coord robot) (make-coordinate :x x :y y))
          (setf (orientation robot) orientation))))

## Testing - Robot Actions

In [ ]:
(defparameter *world* (initialize-world))

In [ ]:
(coord (robot *world*))

In [ ]:
(orientation (robot *world*))

In [ ]:
(coord (first (treasures *world*)))

In [ ]:
(trunk (robot *world*))

In [ ]:
(aref (trunk (robot *world*)) 0)

In [ ]:
(gethash :RED (depots *world*))

In [ ]:
(let* ((robot-coords (coord (robot *world*)))
       (robot-x (coordinate-x robot-coords))
       (robot-y (coordinate-y robot-coords))
       (robot-orientation (orientation (robot *world*))))
      (move (robot *world*) robot-x robot-y robot-orientation)
      (move (robot *world*) robot-x robot-y :NORTH)
      (move (robot *world*) robot-x robot-y :SOUTH)
      (move (robot *world*) robot-x robot-y :EAST)
      (move (robot *world*) robot-x robot-y :WEST))

## Assignment Code - Pathfinding

This part does depth-first-search to find a step-by-step path between the robot and its goal. One steo in that path is a triple of x, y and orientation, like `(3 5 :NORTH)`. A path complete path is a sequence of consecutive steps, that the robot can follow. This is an example path for a robot standing in `(1 3 :EAST)` to find a treasure in `(1 4)`:

`((1 3 :EAST) (1 3 :SOUTH) (1 3 :WEST))`

Another possible path is:

`((1 3 :EAST) (1 3 :NORTH) (2 3 :NORTH) (2 3 :WEST) (2 4 :WEST) (2 5 :WEST)
 (2 6 :WEST) (2 6 :SOUTH) (1 6 :SOUTH) (1 6 :EAST) (1 5 :EAST))`
 
In both cases the robot starts at its current position and ends facing the target entity.

In [ ]:
(defgeneric goal-reached (x y orientation goal)
  (:documentation "Checks if the coordinates and orientation are facing the given goal entity.")
  (:method (x y orientation (goal entity))
    ;; TODO implement
    T))

In [ ]:
(defun turn (orientation direction)
  "Returns the orientation after turning left or right.
    ORIENTATION is a keyword, like the orientation of a robot.
    DIRECTION is either :LEFT or :RIGHT."
  ;; TODO implement
  )

In [ ]:
(defun forward (x y orientation)
  "Returns the position and orientation after moving into ORIENTATION direction.
    Returns x, y and orientation as multiple VALUES."
  ;; TODO implement
  )

In [ ]:
(defun in-path (x y orientation path)
  "Checks if the given x, y and orientation is in the given path,
    where path is a list of triples (x y orientation)."
    ;; TODO implement
    )

In [ ]:
(defgeneric find-path (x y orientation goal &optional path)
  (:documentation "Recursively constructs a `path' from given x, y and orientation, up until the `goal'.
                  Returns the path as a list of (x y orientation) entries. The path ends when facing the goal.
                  This is a depth-first-search approach.")
  (:method (x y orientation (goal entity) &optional path)
           ;; TODO implement
           ))

In [ ]:
;; TODO implement FIND-PATH extra
;; Embed the FIND-PATH method in a closure, like for the MOVE method,
;; and define a local variable.
;; Increase the value of the variable every time a step hits a dead end.

## Testing - Pathfinding

In [ ]:
(defparameter *world* (initialize-world))

In [ ]:
(treasures *world*)

In [ ]:
(coord (robot *world*))

In [ ]:
(orientation (robot *world*))

In [ ]:
;; Test find-path function to treasure
(let* ((robot-coords (coord (robot *world*)))
       (robot-x (coordinate-x robot-coords))
       (robot-y (coordinate-y robot-coords))
       (robot-orientation (orientation (robot *world*)))
       (goal-treasure (nth 1 (treasures *world*)))) ;; change this to test other treasures
  (find-path robot-x robot-y robot-orientation goal-treasure))

In [ ]:
(follow-path (robot *world*) *)

In [ ]:
(collect-treasure (robot *world*))

In [ ]:
(gethash :RED (depots *world*))

In [ ]:
;; Test find-path function to depot
(let* ((robot-coords (coord (robot *world*)))
       (robot-x (coordinate-x robot-coords))
       (robot-y (coordinate-y robot-coords))
       (robot-orientation (orientation (robot *world*)))
       (goal-depot (gethash :RED (depots *world*))))
  (find-path robot-x robot-y robot-orientation goal-depot))

In [ ]:
(follow-path (robot *world*) *)

In [ ]:
(deposit-treasure (robot *world*))

In [ ]:
(discover-world *world*)

The following functions are already compiled.

In [ ]:
(defgeneric follow-path (robot path)
  (:documentation "After calculating the path with find-path,
move along the given path."))

(defmethod follow-path ((robot robot) path)
  (mapcar (lambda (step)
                  (destructuring-bind (x y o) step
                      (move robot x y o)))
                  path))

(defgeneric test-path (world)
  (:documentation "Go to a random treasure in the world and put it to the right depot.")
  (:method ((world treasure-world))
    (let* ((treasure (nth (random (length (treasures world))) (treasures world)))
           (depot (gethash (color treasure) (depots world)))
           (treasure-path (find-path (coordinate-x (coord (robot world)))
                                     (coordinate-y (coord (robot world)))
                                     (orientation (robot world))
                                     treasure)))
      (when treasure-path
        (follow-path (robot world) treasure-path)
        (collect-treasure (robot world))
        (follow-path (robot world) (find-path (coordinate-x (coord (robot world)))
                                              (coordinate-y (coord (robot world)))
                                              (orientation (robot world))
                                              depot))
        (deposit-treasure (robot world))))))

(defmethod discover-world ((world treasure-world))
  (test-path world))